# Dfrac 
Script to calculate the Dfrac of N2H+... 

from reproject import reproject_interp
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.units as u
import astropy.constants as const

In [2]:
hdu_n2dp = fits.open('./data/n2dp_mom0.fits')[0]
hdu_n2dp_err = fits.open('./data/n2dp_mom0err.fits')[0]
hdu_n2hp = fits.open('./data/n2hp_mom0_reprojected.fits')[0]
hdu_n2hp_err = fits.open('./data/n2hp_mom0err_reprojected.fits')[0]

In [3]:
def compute_column_density(W, dW, nu, A_ul, E_u, B, T_ex=10*u.K):
    """
    Compute the total column density in the optically thin, LTE limit using Astropy units.
    
    Parameters
    ----------
    W : Quantity array
        Integrated intensity map in units of K km/s.
    dW : Quantity array
        Error map corresponding to W in units of K km/s.
    nu : Quantity
        Line frequency (e.g., in GHz or Hz).
    A_ul : Quantity or float
        Einstein A coefficient (s^-1).
    E_u : Quantity
        Upper-level energy (in Kelvin).
    B : Quantity
        Rotational constant (e.g., in GHz or Hz).
    T_ex : Quantity, optional
        Excitation temperature (default is 10 K).
        
    Returns
    -------
    N_tot : Quantity
        Total column density map in units of cm^-2.
    dN_tot : Quantity
        Error in the column density (cm^-2).
    """
    # Ensure frequency and rotational constant are in Hz.
    nu = nu.to(u.Hz)
    B = B.to(u.Hz)
    
    # Use Astropy constants in cgs units.
    h = const.h.cgs       # erg s
    k = const.k_B.cgs     # erg/K
    c = const.c.cgs       # cm/s
    
    # Approximate the partition function for a linear rotor: Q_rot = (k T_ex)/(h B)
    Q_rot = (k * T_ex) / (h * B)
    
    # For the J = 1 level, the statistical weight is g_u = 2J+1 = 3.
    g_u = 3.0

    # Convert integrated intensity from K km/s to K cm/s.
    # 1 km/s = 1e5 cm/s.
    W_cgs = W.to(u.K * (u.cm/u.s))
    
    # Compute the factor that converts W to the upper-level column density.
    factor = (8 * np.pi * nu**3) / (c**3 * A_ul)
    
    # Upper-level column density, N_u, in units of cm^-2.
    N_u = factor * W_cgs
    
    # Total column density assuming LTE and optically thin emission:
    N_tot = N_u * (Q_rot / g_u) * np.exp(E_u / T_ex)
    
    # Propagate the error from W (all other parameters are assumed exact).
    dN_tot = N_tot * (dW / W)
    
    return N_tot.decompose(), dN_tot.decompose()

def compute_deuterium_fraction(N_N2D, dN_N2D, N_N2H, dN_N2H):
    """
    Compute the deuterium fraction (N₂D⁺/N₂H⁺) and its uncertainty.
    
    Parameters
    ----------
    N_N2D : Quantity
        Column density for N₂D⁺ (cm^-2).
    dN_N2D : Quantity
        Uncertainty in N₂D⁺ (cm^-2).
    N_N2H : Quantity
        Column density for N₂H⁺ (cm^-2).
    dN_N2H : Quantity
        Uncertainty in N₂H⁺ (cm^-2).
        
    Returns
    -------
    Dfrac : Quantity
        Deuterium fraction (dimensionless).
    dDfrac : Quantity
        Uncertainty in the deuterium fraction (dimensionless).
    """
    Dfrac = N_N2D / N_N2H
    dDfrac = Dfrac * np.sqrt((dN_N2D / N_N2D)**2 + (dN_N2H / N_N2H)**2)
    return Dfrac.decompose(), dDfrac.decompose()

# --- Updated constants for the J=1-0 transitions ---

# For N₂H⁺ (1-0)
nu_N2H = 93.173777 * u.GHz
A_N2H  = 3.62e-5 * u.s**-1
E_u_N2H = 4.47 * u.K
B_N2H  = 46.5869 * u.GHz

# For N₂D⁺ (1-0)
nu_N2D = 77.109 * u.GHz
A_N2D  = 2.95e-5 * u.s**-1  # approximate updated value
E_u_N2D = 3.70 * u.K
B_N2D  = 38.5545 * u.GHz

# Assumed excitation temperature
T_ex = 10 * u.K

# --- Example usage ---
# Here W_N2H, dW_N2H, W_N2D, and dW_N2D should be provided as Quantity arrays,
# e.g., obtained from your FITS files (with units of K km/s).
# For example:
W_N2H = hdu_n2hp.data * u.K * u.km/u.s
dW_N2H = hdu_n2hp_err.data * u.K * u.km/u.s
W_N2D = hdu_n2dp.data * u.K * u.km/u.s
dW_N2D = hdu_n2dp_err.data * u.K * u.km/u.s

# Compute column densities and uncertainties.
N_N2H, dN_N2H = compute_column_density(W_N2H, dW_N2H, nu_N2H, A_N2H, E_u_N2H, B_N2H, T_ex)
N_N2D, dN_N2D = compute_column_density(W_N2D, dW_N2D, nu_N2D, A_N2D, E_u_N2D, B_N2D, T_ex)

# Compute the deuterium fraction and its uncertainty.
Dfrac, dDfrac = compute_deuterium_fraction(N_N2D, dN_N2D, N_N2H, dN_N2H)